In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import numpy as np
from myprotos import string_int_label_map_pb2
##把class裡面的字串分割
from google.protobuf import text_format
import time


In [2]:
##抓取鏡頭初始化0代表電腦上的第一個鏡頭
MyCam = cv2.VideoCapture(0)

In [3]:
##建立字典查詢Class對應的名稱
def LoadLabelmap(LabelMapPath, UseDisplayname=True):
    with open(LabelMapPath) as f:
        Labelmap_String = f.read()
    #print('原本分割前的\n',Labelmap_String)
    LabelMap =string_int_label_map_pb2.StringIntLabelMap()
    ##text_format.Merge(字串,google的字串處理)
    text_format.Merge(Labelmap_String,LabelMap)
    #print('分割後的字串\n',LabelMap.item[0])

    ##新增字典編號當key查詢文字
    LabelmapDict={}
    ##每個編號對應的文字
    for item in LabelMap.item:
        #print(str(item.id)+ " " + item.display_name)
        if(UseDisplayname == True):
            LabelmapDict[item.id]=item.display_name
        else:
            LabelmapDict[item.id]=item.name
    return LabelmapDict
TestLabelMapDict = LoadLabelmap(r'data/mscoco_label_map.pbtxt',True)


In [4]:
#讀取模型(ssd_mobilenet_v1_coco_2018_01_28.tar)
model = tf.saved_model.load(r'model\saved_model')

##輸入與輸出節點(signatures簽章動作)
#print(model.signatures)
model = model.signatures['serving_default']
##輸入
#print(model.inputs)

##輸出
#print(model.outputs)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [5]:
#讀取圖型
#openCV
                          
##抓取鏡頭圖片
while(MyCam.isOpened()):
    ##開始記錄時間
    StartTime =time.time()
    
    ##在鏡頭上抓取成功ret會回傳Ture失敗就是False
    ret,cvimg=MyCam.read()
    ##轉換前(註解轉換後可看不一樣的圖)
    plt.imshow(cvimg)
    ##cv2的色彩是BGR跟其他相反的所以要正常顯示要轉換
    cvimgRGB = cv2.cvtColor(cvimg,cv2.COLOR_BGR2RGB)
    ##轉換後
    #plt.imshow(cvimgRGB)
    ##存放的格式
    #print("存放的格式",type(cvimg))
    #使用模型推理
    input_tensor = tf.convert_to_tensor(cvimgRGB)
    #print("轉換前3維陣列:",input_tensor.shape)
    input_tensor = input_tensor[tf.newaxis,...]
    #print("轉換後4維陣列:",input_tensor.shape)
    out = model(input_tensor)
    #print(out)
    #匯出結果
    ##檢測數量 = out['num_detections'][第幾張圖片]
    ##檢測物件的類別 = out['detection_classes'][第幾張圖片][第幾個框檢測結果]  //如果有2個物件就選0或1
    ##檢測物件信心指數=out['detection_scores'][第幾張圖片][第幾個框檢測結果]
    ##檢測物件的框座標 = out['detection_boxes'][第幾張圖片][第幾個檢測結果][4個座標] // 4個座標為正規化(0~1)的分別為:y1,x1,y2,x2
    #print("檢測到的數量:",out['num_detections'][0])

##    for i in range(int(out['num_detections'][0])):
##        print("第",i,"個準確度是:",out['detection_scores'][0][i])
##        print("第",i,"個座標是:",out['detection_boxes'][0][i])
    ##把4個正規化的座標轉換成像素 數值X長或寬
    for i in range(int(out['num_detections'][0])):
        ClassID = int(out['detection_classes'][0][i])
        Score = float(out['detection_scores'][0][i])
        ##過濾掉準確度低於0.5的
        if(Score >0.5):
            y1 =int(out['detection_boxes'][0][i][0] * cvimgRGB.shape[0])
            x1 =int(out['detection_boxes'][0][i][1] * cvimgRGB.shape[1])
            y2 =int(out['detection_boxes'][0][i][2] * cvimgRGB.shape[0])
            x2 =int(out['detection_boxes'][0][i][3] * cvimgRGB.shape[1])
            ##把框框座標畫出來
            ##cv2.rectangle(目標圖形,起始座標,終點座標,顏色,thickness=框線寬度)
            cv2.rectangle(cvimg,(x1,y1),(x2,y2),(0,255,0),thickness=3)
            ##標示文字
            info = 'Class:{} Score:{}'.format(TestLabelMapDict[ClassID],round(Score,2))
            ##cv2.putText(目標圖片,文字,(x,y),字型,字體大小縮放,顏色(B,G,R),字體粗細)
            cv2.putText(cvimg,info,(x1+5,y1+30),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,100),2)
    EndTime =time.time()
    ##計算處理的所需時間
    Dtime = EndTime - StartTime
    cv2.putText(cvimg,str(int(Dtime*1000))+'ms',(0+5,0+30),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,100),2)
    cv2.imshow('MyCam',cvimg)
    ##如果按下鍵盤上的q鍵離開鏡頭視窗
    if cv2.waitKey(1)==ord('q'):
        break
    
##把鏡頭程式關閉
MyCam.release()
cv2.destroyAllWindows()

Error in callback <function flush_figures at 0x000001D957E564C0> (for post_execute):


KeyboardInterrupt: 